In [1]:
# !pip install geopy

     |████████████████████████████████| 122kB 3.9MB/s eta 0:00:01


In [1]:
from geopy.geocoders import GoogleV3
geolocator = GoogleV3(api_key="")

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
website = requests.get('https://www.bikecollectives.org/wiki/Community_Bicycle_Organizations')
print(website)

<Response [200]>


In [4]:
soup = BeautifulSoup(website.content, 'html.parser')

In [62]:
tst = soup.find_all('h2',text='United States')[0].next_sibling
ind = 0
state = ''
to_store = []
while True:
    tst = tst.next_sibling
    if tst.name=='h2':
        break
    ind += 1
    if (tst.name=='h3'):
        state = tst.text
        continue
    if (tst.name=='ul'):
        for item in tst.find_all('li'):
            if not item.div and item.text != '':
                name = item.text.split('(')[0].strip()
                city = ''
                if '(' in item.text:
                    city = item.text.split('(')[1].replace(')','').strip()
                url = ''
                if (item.a):
                    url = 'https://www.bikecollectives.org'+item.a['href']
                to_store.append({'name': name,
                                 'city': city,
                                 'state': state,
                                 'search_string': f'{name}, {city}, {state}',
                                 'url':url})
                            


In [64]:
len(to_store)

243

In [65]:
for w in to_store:
    location = geolocator.geocode(w['search_string'])
    if location:
        w['geocode']=location[1]
        w['lat']=location[1][0]
        w['lon']=location[1][1]

In [66]:
import pandas as pd

In [67]:
pd.DataFrame(to_store).to_csv('bike_coops.csv')

In [68]:
import json

In [70]:
feature_dict = [
    {
      "type": "Feature",
      "properties": {
        "_umap_options": {
          "iconClass": "Drop",
          "iconUrl": "/uploads/pictogram/bicycle-24-white.png",
          "color": "LimeGreen"
        },
        "description": "",
        "name": f"[[{i['url']}|{i['name']}]]" if 'url' in i else i['name']
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          i['lon'],
          i['lat']
        ]
      }
    } for i in to_store if 'lat' in i]

with open("bike_coops.geojson", "w") as outfile:
    json.dump({"type": "FeatureCollection", "features": feature_dict}, outfile, indent=4)
